# Executing Prompts

Marvin makes executing one-off `task` or `chain` patterns dead simple. 

### Running a `task`

Once you have a prompt defined, fire it off with your chosen LLM asyncronously like so:

In [2]:
from marvin.prompts.library import System, User, ChainOfThought
from marvin.engine.language_models import chat_llm
from typing import Optional


class ExpertSystem(System):
    content: str = (
        "You are a world-class expert on {{topic}}. "
        "When asked questions about {{topic}}, you answer correctly. "
        "You only answer questions about {{topic}}. "
    )
    topic: Optional[str]


class Tutor(System):
    content: str = (
        "When you give an answer, you modulate your response based on the "
        "inferred knowledge of the user. "
        "Your student's name is {{name}}. "
    )
    name: str = "not provided"


model = chat_llm()

response = await model(
    (
        ExpertSystem()
        | Tutor()
        | User(
            "I heard that there are types of geometries when the angles don't add up to"
            " 180?"
        )
        | ChainOfThought()
    ).render(topic="geometry", name="Adam")
)

print(response.content)

Yes, you are correct! In traditional Euclidean geometry, the angles of a triangle always add up to 180 degrees. However, there are indeed other types of geometries where this is not the case. One such example is non-Euclidean geometry, which includes hyperbolic and elliptic geometries. In hyperbolic geometry, the angles of a triangle add up to less than 180 degrees, while in elliptic geometry, the angles add up to more than 180 degrees. These non-Euclidean geometries have their own unique properties and are studied in mathematics and physics.


### Running a `chain`

Of course, some applications require LLMs to run in an iterated loop so that it can deduce
its next actions and take them. We've got you covered. Import an Executor (or create your own) and hit start.

<div class="admonition abstract">
  <p class="admonition-title">Jargon Alert!</p>
  <p>
    An `executor` here is a fancy phrase for a `while` or `for` loop under the context of a conversation with an LLM.
    Below, `OpenAIExectutor` is a bare-bones implementation of 'if you get back a function call, call it, and pass the 
    answer back'. 
  </p>
</div>

In [ ]:
from marvin.prompts.library import System, ChainOfThought, User
from marvin.engine.executors import OpenAIFunctionsExecutor
from marvin.prompts import render_prompts


def write_code(language: str, code: str) -> str:
    """A function that writes code in `language` to accomplish task"""


response = await OpenAIFunctionsExecutor(functions=[write_code]).start(
    prompts=render_prompts(
        System(content="You're an expert on {{subject}}.")
        | User(
            content="I need to know how to write a function in {{subject}} to {{task}}"
        )
        | ChainOfThought(),  # Tell the LLM to think step by step
        {"subject": "python", "task": "calculate the nth fibonacci number"},
    )
)

print(response[-1].content)

In [ ]:
""" 
Here is a Python function that calculates the nth Fibonacci number:

```python
def fibonacci(n):
    if n <= 0:
        return "Invalid input"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```

To use this function, you can simply call it with the desired value of `n`:

```python
result = fibonacci(5)
print(result)  # Output: 3
```

This will calculate and print the 5th Fibonacci number, which is 3.
"""